In [1]:
import os
import matplotlib.pyplot as plt
import neurolib.optimize.exploration.explorationUtils as eu
import numpy as np
from neurolib.models.multimodel import MultiModel
from neurolib.models.multimodel.builder import HopfNetwork
from neurolib.optimize.exploration import BoxSearch
from neurolib.utils.functions import getPowerSpectrum
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.utils.stimulus import ZeroInput

import neurolib.utils.pypetUtils as pu

from hopf_cfc_network import SlowFastHopfNetwork

import h5py
import pandas as pd

plt.style.use("seaborn")
plt.rcParams['image.cmap'] = 'plasma'

# w instrinsic
# 0.003 ~ 0.5Hz
# 0.005 ~ 0.8Hz
# 0.01 ~ 2Hz
# 0.05 ~ 8Hz
# 0.06 ~ 10Hz
# 0.08 ~ 12Hz
# 0.1 ~ 17Hz
# 0.2 ~ 32Hz
# 0.3 ~ 50Hz

In [2]:
DURATION = 20.0 * 1000  # ms
DT = 0.1  # ms

model = MultiModel(
    SlowFastHopfNetwork(
        number_of_slow_fast_units=1,
        slow_to_fast_connection=0.0,
        fast_to_slow_connection=0.0,
    )
)

model.params["duration"] = 20.0 * 1000
model.params["sampling_dt"] = DT
model.params["dt"] = 0.01  # ms - lower dt for numba backend
# numba backend is the way to go in exploration - much faster
model.params["backend"] = "numba"

# manually add params you want to change during exploration, btw model.params is just a dictionary, so you can add whatever :)
model.params["slow_to_fast"] = 0.0
model.params["fast_to_slow"] = 0.0
model.params["bifurcation_param_slow"] = 0.25
model.params["bifurcation_param_fast"] = 0.25
model.params["frequency_slow"] = 0.04
model.params["frequency_fast"] = 0.2

In [3]:
parameters = ParameterSpace(
    {
        "slow_to_fast": np.linspace(0.0, 7.0, 5),
        "fast_to_slow": np.linspace(0.0, 7.0, 5),
        "bifurcation_param_slow": np.linspace(0.0,1,5),
        "bifurcation_param_fast": np.linspace(0.0,1,5),
        "frequency_slow":np.linspace(0.01,0.06,5),
        "frequency_fast":np.linspace(0.1,0.25,5)
    },
    allow_star_notation=True,
    kind="grid",
)

In [4]:
def modulation_index(P):
    """
    MI = KL(P,U)/log(n), where U is a uniform distribution.
    :param P: mean amplitude for each phase bin, shape (N_BINS,)
    :return modulation index, quantitative measure of phase amplitude coupling
    """
    n_bins = P.shape[0]
    try:
        assert np.isclose(np.sum(P), 1)
    except AssertionError:
        print("Please normalize P!")

    kl = np.log(n_bins) + np.sum(P[P!=0] * np.log(P[P!=0]))
    return kl / np.log(n_bins)


def evaluateSimulation(traj):
    # get model with parameters for this run
    model = search.getModelFromTraj(traj)
    # extract stuff you want
    s_f_conn = model.params["slow_to_fast"]
    f_s_conn = model.params["fast_to_slow"]

    # here you can either manually edit connectivity matrix or init again...
    # example of manual edit - easy for single slow-fast unit with 2x2 conn. matrix
    model.params["*connectivity"] = np.array([[0.0, f_s_conn], [s_f_conn, 0.0]])
    model.params["*0.HopfMass_0.a"] = model.params["bifurcation_param_slow"]
    model.params["*1.HopfMass_0.a"] = model.params["bifurcation_param_fast"]
    model.params["*0.HopfMass_0.w"] = model.params["frequency_slow"]
    model.params["*1.HopfMass_0.w"] = model.params["frequency_fast"]

    # example of init again - now commented
    #     model = MultiModel(
    #         SlowFastHopfNetwork(
    #             number_of_slow_fast_units=1,
    #             slow_to_fast_connection=s_f_conn,
    #             fast_to_slow_connection=f_s_conn,
    #         )
    #     )

    #     model.params["duration"] = 20.0 * 1000
    #     model.params["sampling_dt"] = DT
    #     model.params["dt"] = 0.01
    #     model.params["backend"] = "numba"
    #     model.params["slow_to_fast"] = 0.0
    #     model.params["fast_to_slow"] = 0.0

    model.run()

    n_bins = 18

    phase_slow = np.arctan2(model.y[0, :], model.x[0, :])
    amp_fast = np.sqrt(model.x[1, :] ** 2 + model.y[1, :] ** 2)
    binned_phase = np.digitize(phase_slow, bins=np.linspace(-np.pi, np.pi, n_bins + 1))
    mean_bin_amp = np.zeros(n_bins + 1)  # in theory index of bins goes from 0 to N_BINS
    for bin_idx in np.unique(binned_phase):
        mean_bin_amp[bin_idx] = np.mean(amp_fast[binned_phase == bin_idx])

    mean_bin_amp = mean_bin_amp[
        1:
    ]  # because in theory there could be stuff that is smaller than -pi, then actually the interval between -pi and the next bin has index 1.
    # normalize the mean amplitude in each bin
    mean_bin_amp = mean_bin_amp / np.sum(mean_bin_amp)
    mi = modulation_index(mean_bin_amp)


    freq_slow, pow_slow = getPowerSpectrum(model.x.T[:, 0], dt=0.1, maxfr=40, spectrum_windowsize=1)
    freq_fast, pow_fast = getPowerSpectrum(model.x.T[:, 1], dt=0.1, maxfr=40, spectrum_windowsize=1)

    max_freq_fast = freq_fast[np.argmax(pow_fast)]
    max_freq_slow = freq_slow[np.argmax(pow_slow)]


    result_dict = {
        "modulation_index": mi,
        "peak_freq_fast": max_freq_fast,
        "peak_freq_slow": max_freq_slow
                   }

    search.saveToPypet(result_dict, traj)

In [5]:
search = BoxSearch(
    model=model,
    evalFunction=evaluateSimulation,
    parameterSpace=parameters,
    filename="Hopf_conn_extended_exploration.hdf",
    ncores=6,
)

MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `./data/hdf/Hopf_conn_extended_exploration.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.
MainProcess root INFO     Number of parameter configurations: 15625
MainProcess root INFO     BoxSearch: Environment initialized.


In [6]:
pu.getTrajectorynamesInFile("data/hdf/Hopf_conn_extended_exploration.hdf")

['results-2022-04-22-17H-50M-00S']

In [7]:
#search.run()

In [8]:
search.loadResults()

MainProcess root INFO     Loading results from ./data/hdf/Hopf_conn_extended_exploration.hdf
MainProcess root INFO     Analyzing trajectory results-2022-04-22-17H-50M-00S
MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `./data/hdf/Hopf_conn_extended_exploration.hdf`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Loading trajectory `results-2022-04-22-17H-50M-00S`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Loading branch `config` in mode `2`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Loading branch `parameters` in mode `2`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Loading branch `results` in mode `1`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Processed 55283 nodes in 0m20s (2759.03 nodes/s).
MainProcess root INFO     Creating `dfResults` dataframe ...
MainProcess root INFO     Loading all results to `results` dictionary ...
100%|██████████| 15625/15625 [01:05

In [9]:
search.dfResults

,slow_to_fast,fast_to_slow,bifurcation_param_slow,bifurcation_param_fast,frequency_slow,frequency_fast,peak_freq_slow,peak_freq_fast,modulation_index
0,0.0,0.0,0.0,0.0,0.01,0.1000,6.0,32.0,1.323199e-08
1,0.0,0.0,0.0,0.0,0.01,0.1375,6.0,32.0,1.323199e-08
2,0.0,0.0,0.0,0.0,0.01,0.1750,6.0,32.0,1.323199e-08
3,0.0,0.0,0.0,0.0,0.01,0.2125,6.0,32.0,1.323199e-08
4,0.0,0.0,0.0,0.0,0.01,0.2500,6.0,32.0,1.323199e-08
...,...,...,...,...,...,...,...,...,...
15620,7.0,7.0,1.0,1.0,0.06,0.1000,0.0,0.0,7.716259e-01
15621,7.0,7.0,1.0,1.0,0.06,0.1375,0.0,0.0,7.716259e-01
15622,7.0,7.0,1.0,1.0,0.06,0.1750,0.0,0.0,7.716259e-01
15623,7.0,7.0,1.0,1.0,0.06,0.2125,0.0,0.0,7.716259e-01
